# A Yosys Plugin for Checking Graph Isomorphism Between Two Paths

## Introduction
This Yosys plugin checks for graph isomorphism between two paths in a digital circuit design. It takes two starting signals ("-from_a" and "-from_b") and two ending signals ("-to_a" and "-to_b") as input, and compares the paths traversed from the starting signals to the ending signals to determine if they are isomorphic.

## Understanding Yosys and Graph Isomorphism
Yosys is an open-source framework for digital circuit synthesis and optimization. It provides a collection of tools for parsing, analyzing, optimizing, and manipulating digital circuit designs.

Graph isomorphism is a concept in graph theory where two graphs are considered isomorphic if there exists a bijective mapping between their vertices and edges, preserving the graph structure. In the context of digital circuits, graph isomorphism can be used to compare the structural similarity of two paths or subcircuits.

## Plugin Functionality
The plugin performs the following tasks:

1. Parses the "-from_a", "-from_b", "-to_a", and "-to_b" arguments to specify the starting and ending signals for the isomorphism check.
2. Operates on the selected modules in the circuit design.
3. Builds a directed graph representation of the circuit, considering signals as nodes and cell connections as edges.
4. Performs a topological sorting of the graph to ensure proper traversal order.
5. Traverses the paths from the starting signals to the ending signals using a depth-first search (DFS) algorithm.
6. Compares the traversed paths and reports whether they are isomorphic or not.

Here's a flowchart representing the plugin's execution flow:

```
Parse "-from_a", "-from_b", "-to_a", "-to_b" arguments
         |
         v
Select modules in the circuit design
         |
         v
Build directed graph representation
         |
         v
Perform topological sorting
         |
         v
Traverse paths using DFS
         |
         v
    Paths isomorphic?
    /        \
   /          \
  Yes         No
  |            |
  v            v
Report     Report paths
isomorphic  not isomorphic
```

## Code Walkthrough
Key components of the plugin's code:

```cpp
struct GraphIsomorphismWorker {
    // ...
    void topological_sort() {
        // ...
        std::function<void(SigBit)> dfs = [&](SigBit bit) {
            // ...
            if (bit2bits.count(bit) > 0) {
                for (auto &p : bit2bits.at(bit))
                    dfs(p.first);
            }
            // ...
            topo_order.push_back(bit);
        };
        // ...
    }

    bool runner(SigBit bit, Cell *via, vector<string> &path) {
        // ...
        if (bit2bits.count(bit) > 0) {
            for (auto &it : bit2bits.at(bit))
                if (runner(it.first, it.second, path))
                    return true;
        }
        // ...
    }

    void run() {
        // ...
        topological_sort();

        for (auto bit : topo_order) {
            if (bit == from_bit_a)
                if (!runner(bit, nullptr, path_a)) {
                    // ...
                }

            if (bit == from_bit_b)
                if (!runner(bit, nullptr, path_b)) {
                    // ...
                }
        }
        // ...
    }
};
```

The `GraphIsomorphismWorker` class contains the core functionality of the plugin:

1. The `topological_sort()` method performs a topological sorting of the circuit graph using a depth-first search (DFS) algorithm. It ensures that signals are processed in the correct order based on their dependencies.

2. The `runner()` method traverses the paths from the starting signals to the ending signals using a recursive DFS approach. It builds the paths by following the connections between signals and cells.

3. The `run()` method orchestrates the overall execution of the plugin. It calls `topological_sort()` to obtain the proper traversal order and then invokes `runner()` for each starting signal ("-from_a" and "-from_b"). It compares the traversed paths and reports whether they are isomorphic or not.

The plugin also includes error handling and validation checks to ensure that the input signals are valid and exist in the selected modules.

## Example
Using the following circuit:

In [1]:
!cat top.sv

module complex_paths(
    input clk,
    input input1,
    input input2,
    output reg output1,
    output reg output2
);

wire [4:0] path1_intermediates;
wire [4:0] path2_intermediates;
wire path1_dff_out, path2_dff_out;

// Path 1 - Sequence of operations
assign path1_intermediates[0] = input1 ^ input2;
assign path1_intermediates[1] = path1_intermediates[0] & input1;
assign path1_intermediates[2] = path1_intermediates[1] | input2;
dff dff1(.clk(clk), .d(path1_intermediates[2]), .q(path1_dff_out));
assign path1_intermediates[3] = path1_dff_out & ~input1;
assign path1_intermediates[4] = path1_intermediates[3] ^ input2;

// Path 2 - Similar sequence with a subtle difference
assign path2_intermediates[0] = input1 ^ ~input2;
assign path2_intermediates[1] = path2_intermediates[0] & input1;
assign path2_intermediates[2] = path2_intermediates[1] | input2;
dff dff2(.clk(clk), .d(path2_intermediates[2]), .q(path2_dff_out));
assign path2_intermediates[3] = path2_dff_out & input1;
assign path2_

we will check that `input1` and `output1` are equivalent paths to `input2` and `output2`:

In [2]:
import os

!yosys-config --build iso.so main_opt.cc

if os.path.exists("./iso.so"):
    print("File created successfully!")
else:
    print("File creation failed.")

File created successfully!


In [4]:
!yosys -QT -m iso.so -p "read_verilog -sv ./top.sv; synth -flatten -noabc; debug graphiso -from_a input1 -from_b input2 -to_a output1 -to_b output2; show -enum -long" && cp ~/.yosys_show.dot ./out.dot


-- Running command `read_verilog -sv ./top.sv; synth -flatten -noabc; debug graphiso -from_a input1 -from_b input2 -to_a output1 -to_b output2; show -enum -long' --

1. Executing Verilog-2005 frontend: ./top.sv
Parsing SystemVerilog input from `./top.sv' to AST representation.
Generating RTLIL representation for module `\complex_paths'.
Generating RTLIL representation for module `\dff'.
Successfully finished Verilog frontend.

2. Executing SYNTH pass.

2.1. Executing HIERARCHY pass (managing design hierarchy).

2.1.1. Finding top of design hierarchy..
root of   0 design levels: dff                 
root of   1 design levels: complex_paths       
Automatically selected complex_paths as design top module.

2.1.2. Analyzing design hierarchy..
Top module:  \complex_paths
Used module:     \dff

2.1.3. Analyzing design hierarchy..
Top module:  \complex_paths
Used module:     \dff
Removed 0 unused modules.

2.2. Executing PROC pass (convert processes to netlists).

2.2.1. Executing PROC_CLEA

---
The plugin shows that both paths are not structurally equivalent, marking the first differences and showing the schematic for visual inspection

In [5]:
!pip install ipywidgets
from graphviz import Source
from IPython.display import display, HTML

src = Source.from_file('out.dot')
svg_output = src.pipe(format='svg').decode('utf-8')
html_output = f"<div style='width:800px; height:600px; overflow:auto;'>{svg_output}</div>"
display(HTML(html_output))

Highlighting the differences we can see them clearly.

In [7]:
from graphviz import Source
from IPython.display import display, HTML

src = Source.from_file('isomorphic.dot')
svg_output = src.pipe(format='svg').decode('utf-8')
html_output = f"<div style='width:800px; height:600px; overflow:auto;'>{svg_output}</div>"
display(HTML(html_output))